In [8]:
from bs4 import BeautifulSoup
import glob
import re
import csv
import spacy
import sys
from nltk import sent_tokenize
import pandas as pd
nlp = spacy.load('en_core_web_sm')
keyword_processor = KeywordProcessor(case_sensitive=True)

In [9]:
## 處理特殊狀況的句子
def Filter_Title(sen):
    flag = False
    doc = nlp(sen)
    for token in doc:
        if token.pos_=='VERB':
            flag = True
    return flag

## 前處理(過濾表格、圖、標題)
def Deal_html_return_sen(html_list):
    ## NLTK句子切割
    def nltk_sentence_token(Content):
        Sen= []
        sen_token = sent_tokenize(Content)
        for sen in sen_token:
            if len(sen.split(' '))>=5 and Filter_Title(sen):
                Sen.append(sen)
        return Sen
    ## 處理特殊狀況的句子
    def Filter_Title(sen):
        flag = False
        doc = nlp(sen)
        for token in doc:
            if token.pos_=='VERB':
                flag = True
        return flag
    Sen = []
    for path_html in html_list:
        file = open(path_html , 'r')
        soup = BeautifulSoup(file , 'html.parser')
        file.close()
        tmpp = soup.find_all('p')
        After_filter_title_sen = [i.text.strip() for i in tmpp if i.text and len(i.text.split(' '))>=6 and Filter_Title(i.text)]
        Content = ' '.join(After_filter_title_sen)
        Sen+=nltk_sentence_token(Content)
    return Sen

In [10]:
df_e = pd.read_csv('flair.csv')
df_e[:1]
Ent = []
for ent , label in zip(df_e['Entity'],df_e['islabel']):
    if label == 1:
        Ent.append(ent)
Ent = [i.replace('LLC','').replace('Ltd','').replace('Inc','').replace(',','').replace('N.V','').replace('Co.','').replace('.','').strip() for i in Ent]

In [11]:
for i in Ent:keyword_processor.add_keyword(i)

In [12]:
html_list = glob.glob('xxxx/Broadcom/html/2014*.html')
html_list

['xxxx/Broadcom/html/2014.html', 'xxxx/Broadcom/html/2014-2.html']

In [13]:
Total_sen = Deal_html_return_sen(html_list)

In [14]:
Main_Company = html_list[0].split('/')[1]
Year = html_list[0].split('/')[-1].split('-')[0].replace('.html','')
print(Main_Company)
print(Year)

Broadcom
2014


In [15]:
Tmp = []
for s in Total_sen:
    Ent = keyword_processor.extract_keywords(s)
    for e in Ent:
        Tmp.append((s , Main_Company,e , Year))

In [17]:
import csv
with open('test.sentence.csv','w') as f:
    writer = csv.writer(f)
    writer.writerows([('Sentence','Main','Secondary','Time','Label')])
    writer.writerows(Tmp)

In [11]:
len(Tmp)

12715